<a href="https://colab.research.google.com/github/ICPCfirst/DeepLearningHW2/blob/main/HW2(Generative).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np

np.random.seed(0)
X_train_fpath = "/content/drive/MyDrive/data/X_train"
X_test_fpath = "/content/drive/MyDrive/data/X_test"
Y_train_fpath = "/content/drive/MyDrive/data/Y_train"
output_fpath = "./output_{}.csv"

In [23]:
with open(X_train_fpath) as f:
  next(f)
  X_train = np.array([line.strip("\n").split(",")[1:] for line in f],dtype = float)
with open(X_test_fpath) as f:
  next(f)
  X_test = np.array([line.strip("\n").split(",")[1:] for line in f],dtype = float)
with open(Y_train_fpath) as f:
  next(f)
  Y_train = np.array([line.strip("\n").split(",")[1:] for line in f],dtype = float)
  

In [24]:
def _normalize(X,train = True,specified_column = None,X_mean = None,X_std = None):
  if specified_column == None:
    specified_column = np.arange(X.shape[1])
  if train:
    X_mean = np.mean(X[:,specified_column],0).reshape(1,-1) # meean(0)==压缩行 ；reshape == 必须转化为一行n列 ？？？？？？？是不是重复了
    X_std = np.std(X[:,specified_column]).reshape(1,-1)
  
  X[:,specified_column] = (X[:,specified_column]-X_mean)/(X_std + 1e-8)# 进行归一化，中心化和标准处理化
  return X,X_mean,X_std

In [25]:
def _train_dev_split(X,Y,dev_ratio = 0.25):
  train_size = int(len(X)*(1-dev_ratio))
  return X[:train_size],Y[:train_size],X[train_size:],Y[train_size:]

In [26]:
X_train,X_mean,X_std = _normalize(X_train,train = True)
X_test,_,_ = _normalize(X_test,train = False,specified_column=None,X_mean = X_mean,X_std = X_std)

dev_ratio = 0.1
X_train,Y_train,X_dev,Y_dev = _train_dev_split(X_train,Y_train,dev_ratio = dev_ratio) #自己根据已给数据分开来使程序准确性更高

train_size = X_train.shape[0]
dev_size = X_dev.shape[0]
test_size = X_test.shape[0]
data_dim = X_train.shape[1]

In [27]:
def _shuffle(X, Y):  #洗牌，排序
    # 此函数将两个等长的列表/数组 X 和 Y 混在一起。
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
    return (X[randomize], Y[randomize])    #作用是将行重新打乱。

In [28]:
def _sigmoid(z):            #经典
    # Sigmoid function can be used to calculate probability.
    # To avoid overflow, minimum/maximum output value is set.
    return np.clip(1 / (1.0 + np.exp(-z)), 1e-8, 1 - (1e-8))#np.clip(a,min,max),作用是将所有输出的a都限制在min和max之间。

In [29]:
def _f(X, w, b):
     # 这是逻辑回归函数，由 w 和 b 参数化
     #
     # 论点：
     # X: 输入数据，shape = [batch_size, data_dimension] # 【8，510】
     # w: 权重向量, shape = [data_dimension, ] # 【510，】
     # b: 偏差，标量
     # 输出：
     # X 的每一行被正标记的预测概率，shape = [batch_size, ]
    return _sigmoid(np.matmul(X, w) + b)#二维乘法。  返回n行1列

In [30]:
def _predict(X, w, b):
    # 此函数返回 X 的每一行的真值预测+
    # 通过四舍五入逻辑回归函数的结果。
    return np.round(_f(X, w, b)).astype(np.int)# 这里是取整 取整后容易产生梯度爆炸或溢出，故之后的loss不采用此函数；全精度>混合精度>半精度

In [31]:
def _accuracy(Y_pred, Y_label):# 精度越大越好
    # 这个函数计算预测精度
    acc = 1 - np.mean(np.abs(Y_pred - Y_label))
    return acc   

In [32]:
def _cross_entropy_loss(y_pred, Y_label):
    224 / 5,000
    # 这个函数计算交叉熵。
    #
    # 论点：
    # y_pred：概率预测，浮点向量
    # Y_label：ground truth 标签，布尔向量
    # 输出：
    # 交叉熵，标量
    cross_entropy = -np.dot(Y_label, np.log(y_pred)) - np.dot((1 - Y_label), np.log(1 - y_pred))# np.dot 矩阵乘积
    return cross_entropy

In [33]:
def _gradient(X, Y_label, w, b):
    # 此函数计算交叉熵损失相对于权重 w 和偏差 b 的梯度。
    y_pred = _f(X, w, b)
    pred_error = Y_label - y_pred
    w_grad = -np.sum(pred_error * X.T, 1)  # -E(y_ - y)*X.T T(转置)
    b_grad = -np.sum(pred_error)  #可不可以换成mean
    return w_grad, b_grad


In [ ]:
# 分别计算类别0和类别1的均值
X_train_0 = np.array([x for x, y in zip(X_train, Y_train) if y == 0])  # u0
X_train_1 = np.array([x for x, y in zip(X_train, Y_train) if y == 1])  # u1

mean_0 = np.mean(X_train_0, axis = 0)
mean_1 = np.mean(X_train_1, axis = 0)

#print(X_train_0.shape) #(38784, 510) 有三万多行数据的结果是u0
#print("*"*50) 
#print(X_train_1.shape) #(10046, 510)) 有一万多行数据的结果是u1
#print("*"*50)
#print(mean_0.shape)
#print("*"*50)
#print(mean_1.shape)

In [ ]:
# 分别计算类别0和类别1的协方差
cov_0 = np.zeros((data_dim, data_dim))
cov_1 = np.zeros((data_dim, data_dim))

for x in X_train_0:
    cov_0 += np.dot(np.transpose([x - mean_0]), [x - mean_0]) / X_train_0.shape[0]
for x in X_train_1:
    cov_1 += np.dot(np.transpose([x - mean_1]), [x - mean_1]) / X_train_1.shape[0]

# 共享协方差 = 独立的协方差的加权求和
cov = (cov_0 * X_train_0.shape[0] + cov_1 * X_train_1.shape[0]) / (X_train_0.shape[0] + X_train_1.shape[0])
